# What are we trying to find out

1. Is the [recommended strategy](http://www.lasvegas-how-to.com/blackjack.php) for blackjack better than just playing randomly?
2. Can we do better than the [recommended strategy](http://www.lasvegas-how-to.com/blackjack.php)?

# How to play blackjack

## Objective

- Try to get as close to 21 points as possible without going over
- Hand with the higher points wins
- A hand that opens with 21 (Ace + 10 or J or Q or K) is a blackjack
    - Best possible hand

## Outcomes

- player gets blackjack, dealer didn't
    - player wins
- dealer gets blackjack, player didn't
    - player loses
- player hits and busts
    - player loses
- player hits, dealer hits and busts
    - player wins
- player hits, dealer hits/ dealer >= hard 17
    - player > dealer
        - player wins
    - player < dealer
        - player loses
    - player == dealer
        - draw

## Calculating points

- J, Q, K == 10
- A == 11 if it doesn't cause the hand to bust, otherwise A == 1
- Numerical cards are counted based on the numbers (i.e. 2 is 2 points, 3 is 3 points etc.)

## How does a game end

- Player gets blackjack
- Dealer gets blackjack
- Player hits and busts
- Player finishes hitting, dealer hits and busts
- Player and dealer finish hitting

## Playing the game

- Player and dealer are dealt 2 cards each
    - Alternating between player and dealer
- Both the player's cards are open
- Only the dealer's first card is open
    - If it's an Ace, dealer checks if he has a value of 10 on his covered card
    - If yes, he opens, and the game ends
- The player has a chance to decide whether or not he wants more card (hit)
    - If he has a blackjack, the game ends
    - He hits until he decides that he doesn't want to hit anymore or he busts
    - Whichever is earlier
    - If he busts, the game ends
- The dealer opens his covered card
    - If his total is below 17, he hits
    - If his total is 17, but it's a soft 17, he hits
    - If he busts, the game ends
- The points are compared

# Strategies

## [Recommended strategy](http://www.lasvegas-how-to.com/blackjack.php)

- Stand on 17 or more
- If you have 12-16 points (inclusive), and the dealer is showing 16 or less, hit
    - I believe the assumption is that the dealer’s closed card always has a value of 10
    - So 16 or less, means if their open card is 6 or below
- Always split 8’s
- Double down on 11 if the dealer’s open card is 7 or less
- If you have 11 or lower, always hit
    - It’s not possible for you to bust

Some other tips (which seem to be for slightly more advanced players):

- If you have 17 points and the dealer shows 7 or higher, consider hitting
    - The chance of you busting is very high
    - But if his open card is 7 or higher, and assuming the closed card is 10, you’re going to lose anyway
- If you have 12-16 points (inclusive), and the dealer is showing 2 or 3, consider hitting
- If you have a soft hand (Ace that is counted as 11 points), consider hitting to get a better hand

## Random strategy

Even when playing randomly, we should have some form of strategy

- Stand on a hand of 18 or more
- Always hit on 11 or less
- Between 12 to 17, randomize whether or not to hit

# Risks and assumptions

## Assumptions

- Multiple decks are used
    - 4, 6, or 8
- Only one player and the dealer (for simplicity)
- Continuous shuffling machines used
    - It's as though we're using a fresh deck each time
- Assume no splitting
- Assume no double down
- We are just deciding wins, draws and losses, not amount of bets earned or loss

## Risks

- I am generating my own dataset
    - A lot of checks required to ensure that the dataset is accurate
- Not all casinos use continuous shuffling machines
    - From what I read on the web (and sometimes I just stand there and watch them shuffle)
    - If a deck is used till there's only a certain number of cards left (instead of using a new deck), the analysis is likely to be different
        - There's not an equal probability of drawing each card for each hand
- I can't account for the parts of the strategy where it's "consider hitting"
    - i.e. where player decision is required

# Refining our questions

- Is the recommended strategy better than a random strategy?
    - When playing with the recommended strategy as compared to the random strategy
        - Is our expected proportion of wins bigger?
        - Is our expected proportion of losses smaller?
- Can we do better than the recommended strategy?
    - Is splitting 8's always better than just keeping them?
    - Do we always do better when hitting on 12-16's?
    - Do we always do worse when hitting on 17's and above?

# What data do we need

- Simulating the game
    - What are the chances of getting a card given the cards we can see
        - Dealer open
        - Player card one
        - Player card two
        - Number of decks of cards that we're playing with
    - Deciding when the player should hit
        - Strategy
        - Dealer open
        - Player initial points
    - Deciding who wins
        - Dealer initial points
        - Dealer final points
        - Player initial points
        - Dealer final points
    - Misc (record keeping)
        - Did the dealer hit
        - How many times did the dealer hit
        - Did the player hit
        - How many times did the player hit
        - Player hand
        - Dealer hand
        - Dealer bust
        - Player bust
        
- Answering our questions
    - When playing with the recommended strategy as compared to the random strategy
        - Is our expected proportion of wins bigger?
        - Is our expected proportion of losses smaller?
            - Strategy
            - Player wins
            - Player loses
            - Draw
    - Is splitting 8's always better than just keeping them?
        - Player hand
        - Player wins
        - Player loses
        - Draw
    - Do we always do better when hitting on 12-16's?
        - Player initial points
        - Player wins
        - Player loses
        - Draw
    - Do we always do worse when hitting on 17's and above?
        - Player initial points
        - Player wins
        - Player loses
        - Draw

## Data dictionary

In [2]:
import pandas as pd

In [3]:
pd.set_option("max_colwidth", 0)

In [4]:
data_dictionary = pd.DataFrame([["num_decks", "Int", "Number of decks used"], 
                                ["dealer_open", "Int", "The card we can see"], 
                                ["dealer_initial", "Int", "The dealer's starting points"], 
                                ["dealer_hit", "Binary", "1 - dealer hit, 0 - dealer did not hit"], 
                                ["dealer_num_hits", "Int", "Number of times the dealer hit"], 
                                ["dealer_final", "Int", "The dealer's final points"], 
                                ["dealer_busts", "Binary", "1 - dealer busts, 0 - dealer did not bust"], 
                                ["player_card_one", "Int", "Player's first card"], 
                                ["player_card_two", "Int", "Player's second card"], 
                                ["player_initial", "Int", "The player's starting points"], 
                                ["player_hit", "Binary", "1 - player hit, 0 - player did not hit"], 
                                ["player_num_hits", "Int", "Number of times the player hit"], 
                                ["player_final", "Int", "The player's final points"], 
                                ["player_busts", "Binary", "1 - player busts, 0 - player did not bust"], 
                                ["player_loses", "Binary", "1 - player lost, 0 - player did not lose"], 
                                ["draw", "Binary", "1 - draw, 0 - not a draw"], 
                                ["player_wins", "Binary", "1 - player won, 0 - player did not win"], 
                                ["strategy", "Binary", "1 - recommended, 0 - random"], 
                                ["dealer_hand", "Obj", "List of the dealer's cards, in the order that they were dealt"], 
                                ["player_hand", "Obj", "List of the player's cards, in the order that they were dealt"]], 
                               columns=["Feature", "Type", "Description"])

In [5]:
data_dictionary

,Feature,Type,Description
0,num_decks,Int,Number of decks used
1,dealer_open,Int,The card we can see
2,dealer_initial,Int,The dealer's starting points
3,dealer_hit,Binary,"1 - dealer hit, 0 - dealer did not hit"
4,dealer_num_hits,Int,Number of times the dealer hit
5,dealer_final,Int,The dealer's final points
6,dealer_busts,Binary,"1 - dealer busts, 0 - dealer did not bust"
7,player_card_one,Int,Player's first card
8,player_card_two,Int,Player's second card
9,player_initial,Int,The player's starting points


In [21]:
data_dictionary.to_csv("data_dictionary.csv", index=False)